In [1]:
import os

from bids import BIDSLayout

from glob import glob

from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function

Failed to import duecredit due to No module named 'duecredit'


In [2]:
bids_dir = os.path.join('/home/sirius/BrainHack Geneva 2019','ds-supermri','bids')
output_dir = os.path.join('/home/sirius/BrainHack Geneva 2019','ds-supermri','bids','derivatives','mialsrtk')

subject = 'eyesVD46a'

layout = BIDSLayout(bids_dir)
print(layout)

BIDS Layout: ...k Geneva 2019/ds-supermri/bids | Subjects: 1 | Sessions: 0 | Runs: 6


In [3]:
bg = Node(BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
bg.inputs.base_dir = bids_dir
bg.inputs.subject = subject
bg.inputs.output_query = {'T2ws': dict(suffix='T2w')}

In [4]:
def filterNiftis(paths):
    paths = [p for p in paths if 'nii' in p]
    print("\n\nLoaded T2w images " + str(paths) + "\n\n")
    return paths

filterT2wNii = Node(Function(function=filterNiftis, input_names=["paths"],
                            output_names=["paths"]), name="filterT2wNii")

wf = Workflow(name="bids_demo",base_dir=output_dir)
wf.connect(bg, "T2ws", filterT2wNii, "paths")
res = wf.run()

190323-12:03:31,747 nipype.workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging', 'monitoring']
190323-12:03:31,749 nipype.workflow INFO:
	 Running serially.
190323-12:03:31,750 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.bids_grabber" in "/home/sirius/BrainHack Geneva 2019/ds-supermri/bids/derivatives/mialsrtk/bids_demo/bids_grabber".
190323-12:03:31,753 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")
190323-12:03:31,769 nipype.workflow INFO:
	 [Node] Finished "bids_demo.bids_grabber".
190323-12:03:31,769 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.filterT2wNii" in "/home/sirius/BrainHack Geneva 2019/ds-supermri/bids/derivatives/mialsrtk/bids_demo/filterT2wNii".
190323-12:03:31,771 nipype.workflow INFO:
	 [Node] Outdated cache found for "bids_demo.filterT2wNii".
190323-12:03:31,773 nipype.workflow INFO:
	 [Node] Running "filterT2wNii" ("nipype.interfaces.utility.wrappers.Function")


Loaded T2w i

In [5]:
from traits.api import *
from nipype.utils.filemanip import split_filename

from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

class prepareDockerPathsInputSpec(BaseInterfaceInputSpec):
    local_paths = InputMultiPath(File(desc='input paths', mandatory = True, exists = True))
    local_dir = Directory(mandatory=True)
    docker_dir = Directory('/fetaldata',mandatory=True)

class prepareDockerPathsOutputSpec(TraitedSpec):
    docker_paths = OutputMultiPath(File(desc='docker paths'))

class prepareDockerPaths(BaseInterface):
    input_spec = prepareDockerPathsInputSpec
    output_spec = prepareDockerPathsOutputSpec

    def _run_interface(self,runtime):
        return runtime
           
    def _list_outputs(self):

        outputs = self._outputs().get()
        outputs["docker_paths"] = []
        for p in self.inputs.local_paths:
            p = os.path.join(self.inputs.docker_dir,p.split(self.inputs.local_dir)[1].strip("/"))
            print(p)
            outputs["docker_paths"].append(p)
        return outputs
    

preparePaths = Node(interface=prepareDockerPaths(), name="preparePaths")  

preparePaths.inputs.local_dir = bids_dir
preparePaths.inputs.docker_dir = '/fetaldata'

wf.connect(filterT2wNii, "paths", preparePaths, "local_paths")
res = wf.run()

190323-12:03:31,786 nipype.workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging', 'monitoring']
190323-12:03:31,791 nipype.workflow INFO:
	 Running serially.
190323-12:03:31,792 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.bids_grabber" in "/home/sirius/BrainHack Geneva 2019/ds-supermri/bids/derivatives/mialsrtk/bids_demo/bids_grabber".
190323-12:03:31,795 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")
190323-12:03:31,811 nipype.workflow INFO:
	 [Node] Finished "bids_demo.bids_grabber".
190323-12:03:31,811 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.filterT2wNii" in "/home/sirius/BrainHack Geneva 2019/ds-supermri/bids/derivatives/mialsrtk/bids_demo/filterT2wNii".
190323-12:03:31,813 nipype.workflow INFO:
	 [Node] Outdated cache found for "bids_demo.filterT2wNii".
190323-12:03:31,815 nipype.workflow INFO:
	 [Node] Running "filterT2wNii" ("nipype.interfaces.utility.wrappers.Function")


Loaded T2w i

In [6]:
import subprocess
def run(self, command, env={}, cwd=os.getcwd()):
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.run(command, shell=True,
                               env=merged_env, cwd=cwd, capture_output=True)
    return process

In [7]:
#proc = run(self,'docker run --rm -v "/home/sirius/BrainHack Geneva 2019/ds-supermri/bids":/fetaldata --entrypoint "btkNLMDenoising" sebastientourbier/mialsuperresolutiontoolkit -i /fetaldata/sub-eyesVD46a/anat/sub-eyesVD46a_run-1_T2w.nii.gz -o /fetaldata/sub-eyesVD46a_run-1_T2w_nlm.nii.gz -b 0.1')
#print(proc.stdout)

In [8]:
os.getcwd().replace('/home/sirius/BrainHack Geneva 2019/mialsuperresolutiontoolkit/supermribidsapp','/fetaldata')

'/fetaldata/mialsrtk/example'

In [9]:
from traits.api import *
from nipype.utils.filemanip import split_filename

from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

import nibabel as nib

class BtkNLMDenoisingInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    in_file = File(desc='Input image',mandatory=True,)
    out_postfix = traits.Str("_nlm", usedefault=True)
    weight = traits.Float(0.1,desc='NLM weight (0.1 by default)')

class BtkNLMDenoisingOutputSpec(TraitedSpec):
    out_file = File(desc='Denoised image')

class BtkNLMDenoising(BaseInterface):

    input_spec = BtkNLMDenoisingInputSpec
    output_spec = BtkNLMDenoisingOutputSpec
    
    def _run_interface(self, runtime):
        _, name, ext = split_filename(os.path.abspath(self.inputs.in_file))
        out_file = os.path.join(os.getcwd().replace(self.inputs.bids_dir,'/fetaldata'), ''.join((name, self.inputs.out_postfix, ext)))
        cmd = 'docker run --rm -u {}:{} -v "{}":/fetaldata sebastientourbier/mialsuperresolutiontoolkit btkNLMDenoising -i {} -o {} -b {}'.format(os.getuid(),os.getgid(),self.inputs.bids_dir,self.inputs.in_file,out_file,self.inputs.weight)
        try:
            print('... cmd: {}'.format(cmd))
            run(self, cmd, env={}, cwd=os.path.abspath(self.inputs.bids_dir))
        except:
            print('Failed')
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        _, name, ext = split_filename(os.path.abspath(self.inputs.in_file))
        outputs['out_file'] = os.path.join(os.getcwd(), ''.join((name, self.inputs.out_postfix, ext)))
        return outputs

class MultipleBtkNLMDenoisingInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    input_images = InputMultiPath(File(desc='files to be denoised', mandatory = True))
    weight = traits.Float(0.1)
    out_postfix = traits.Str("_nlm", usedefault=True)
    
class MultipleBtkNLMDenoisingOutputSpec(TraitedSpec):
    output_images = OutputMultiPath(File())

class MultipleBtkNLMDenoising(BaseInterface):
    input_spec = MultipleBtkNLMDenoisingInputSpec
    output_spec = MultipleBtkNLMDenoisingOutputSpec

    def _run_interface(self, runtime):
        for input_image in self.inputs.input_images:
            ax = BtkNLMDenoising(bids_dir = self.inputs.bids_dir, in_file = input_image, out_postfix=self.inputs.out_postfix, weight = self.inputs.weight)
            ax.run()
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['output_images'] = glob(os.path.abspath("*.nii.gz"))
        return outputs
    

nlmDenoise = Node(interface=MultipleBtkNLMDenoising(),base_dir=os.path.join(output_dir,'bids_demo'),name='nlmDenoise')
nlmDenoise.inputs.bids_dir = bids_dir
nlmDenoise.inputs.weight = 0.1
                  
wf.connect(preparePaths, "docker_paths", nlmDenoise, "input_images")
res = wf.run()

190323-12:03:32,276 nipype.workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging', 'monitoring']
190323-12:03:32,279 nipype.workflow INFO:
	 Running serially.
190323-12:03:32,280 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.bids_grabber" in "/home/sirius/BrainHack Geneva 2019/ds-supermri/bids/derivatives/mialsrtk/bids_demo/bids_grabber".
190323-12:03:32,282 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")
190323-12:03:32,297 nipype.workflow INFO:
	 [Node] Finished "bids_demo.bids_grabber".
190323-12:03:32,298 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.filterT2wNii" in "/home/sirius/BrainHack Geneva 2019/ds-supermri/bids/derivatives/mialsrtk/bids_demo/filterT2wNii".
190323-12:03:32,299 nipype.workflow INFO:
	 [Node] Cached "bids_demo.filterT2wNii" - collecting precomputed outputs
190323-12:03:32,300 nipype.workflow INFO:
	 [Node] "bids_demo.filterT2wNii" found cached.
190323-12:03:32,300 nipype.workf